# Pre-process Text Data

This notebook will convert the reddit post text to model-readable features, perform train-test splits, and save to .csv files. Applies varying lags to processed text data for different prediction horizons.

In [1]:
import sys
import os
import re
import time
import json

import datetime as dt
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:

exp_file = '../data/tabular/wsb_posts/cleaned posts.csv'

if os.path.isfile(exp_file):
    print('File found, loading...')
    combined_df = pd.read_csv(exp_file)

else:
    print('File not found, processing data from files...')
    
    def load_posts1(file):
        df = pd.read_csv(file)
        df['created_utc'] = pd.to_datetime(df['created_utc'],
                                           origin = 'unix',
                                           unit = 's')
        df.loc[df['selftext'].isna(), 'selftext'] = ''
        df = df.sort_values('created_utc')
        return df[['created_utc', 'num_comments', 'score', 'title', 'selftext']]
    posts_2018 = load_posts1('../data/tabular/wsb_posts/WSB Posts 2018.csv')



    def extract_wrapper(pattern):
        def extractor(string):
            prog = re.compile(pattern)
            results = [list(elem) for elem in prog.findall(string)]
            if len(results) == 0:
                return results
            elif len(results) == 1:
                return [elem for elem in results[0] if elem.isalpha()]
            else:
                result = results[0]
                for next_res in results[1:]:
                    result = [elem for elem in result if elem.isalpha()]
                    result += next_res
                return result
        return extractor

    # Extact tickers from text
    def extract_tickers(df):
        df = df.copy()
        df['selftext'] = df['selftext'].fillna('')
        patterns = [
            r'\$([A-Z]{1}) +',
            r'\$([A-Z]{2})(\s|[0-9a-z]|\Z|\W)',
            r'\$([A-Z]{3})(\s|[0-9a-z]|\Z|\W)',
            r'\$([A-Z]{4})(\s|[0-9a-z]|\Z|\W)',
            r'(\A|\s)([A-Z]{1}) +',
            r'(\A|\s)([A-Z]{2})(\s|[0-9a-z]|\Z|\W)',
            r'(\A|\s)([A-Z]{3})(\s|[0-9a-z]|\Z|\W)',
            r'(\A|\s)([A-Z]{4})(\s|[0-9a-z]|\Z|\W)'
        ]

        results = [df['title'].apply(extract_wrapper(pattern)) for pattern in patterns]
        results = results + [df['selftext'].apply(extract_wrapper(pattern)) for pattern in patterns]

        new_col = results[0]

        for result in results[1:]:
            new_col += result

        df['tickers'] = new_col
        df['num_tickers'] = df['tickers'].apply(len)
        return df


    extracted_2018 = extract_tickers(posts_2018)
    extracted_2018 = extracted_2018[extracted_2018.num_tickers > 0]

    # validate tickers
    valid_tickers = set(pd.read_csv('../data/tabular/tickers.csv').Ticker.values)
    valid_tickers = valid_tickers.difference({'YOLO', 'IPO'})  # Remove YOLO ticker

    def validate_tickers(df, valid_set):
        df = df.copy()
        df['tickers'] = df['tickers'].apply(
            lambda x: list(set([
                ticker for ticker in x if ticker in valid_set
            ]))
        )
        df['num_tickers'] = df['tickers'].apply(len)
        return df

    validated_2018 = validate_tickers(extracted_2018, valid_tickers)
    validated_2018 = validated_2018[validated_2018.num_tickers > 0]

    # Combine text columns
    def reduce_columns(df):
        df = df.copy()
        df['text'] = df['title']  + '\n' + df['selftext']
        df = df.drop(columns=['title', 'selftext'])
        return df

    reduced_2018 = reduce_columns(validated_2018)


    posts_2019 = load_posts1('../data/tabular/wsb_posts/WSB Posts 2019.csv')
    extracted_2019 = extract_tickers(posts_2019)
    extracted_2019 = extracted_2019[extracted_2019.num_tickers > 0]
    validated_2019 = validate_tickers(extracted_2019, valid_tickers)
    validated_2019 = validated_2019[validated_2019.num_tickers > 0]
    reduced_2019 = reduce_columns(validated_2019)

    def load_posts2():
        prefix = '../data/heirarchical/wsb posts 2019'
        filenames  = [
            'RS_2019-09.json',
            'RS_2019-10.json',
            'RS_2019-11.json',
            'RS_2019-12.json'
        ]
        df = pd.read_json('{}/{}'.format(prefix, filenames[0]))
        for name in filenames[1:]:
            df = pd.concat((df, pd.read_json('{}/{}'.format(prefix, name))))
        df['created_utc'] = pd.to_datetime(df['created_utc'],
                                           origin = 'unix',
                                           unit = 's')
        df.loc[df['selftext'].isna(), 'selftext'] = ''
        df = df.sort_values('created_utc')
        return df[['created_utc', 'num_comments', 'score', 'title', 'selftext']]

    remaining_2019 = load_posts2()

    extracted_remaining = extract_tickers(remaining_2019)
    extracted_remaining = extracted_remaining[extracted_remaining.num_tickers > 0]
    validated_remaining = validate_tickers(extracted_remaining, valid_tickers)
    validated_remaining = validated_remaining[validated_remaining.num_tickers > 0]
    reduced_remaining = reduce_columns(validated_remaining)


    combined_df = pd.concat((reduced_2018, reduced_2019, reduced_remaining))
    combined_df.index = np.arange(combined_df.shape[0])
    combined_df['tickers'] = combined_df.tickers.apply(lambda x: ','.join(x))

    # let's add a couple columns for words that we expect to be important
    combined_df['mentions_option'] = combined_df['text'].apply(lambda x: 'option' in x.lower()).astype(int)
    combined_df['mentions_call'] = combined_df['text'].apply(lambda x: 'call' in x.lower()).astype(int)
    combined_df['mentions_put'] = combined_df['text'].apply(lambda x: 'put' in x.lower()).astype(int)

    # finally, add a hash for the text string so that we can quickly relate data to it later
    combined_df['text_hash'] = pd.util.hash_pandas_object(combined_df['text'])

    combined_df.to_csv(exp_file)

combined_df.head(2)

File found, loading...


,Unnamed: 0,Unnamed: 0.1,created_utc,num_comments,score,tickers,num_tickers,text,mentions_option,mentions_call,mentions_put,text_hash
0,0,0,2018-05-01 00:06:43,24,3,QQQ,1,What kind of option play would you use to prof...,1,0,0,150508729389721599
1,1,1,2018-05-01 00:12:41,12,41,AAPL,1,Tariffs are delayed meaning AAPL calls are gua...,0,1,0,10678551544699203972


Print some samples of post text to get some intuition for the documents in the corpus.

In [12]:
samples = combined_df.text.sample(4)
for _, sample in samples.iteritems():
    print(sample)
    print('-----------')

Would this work?
Let’s say I buy 1,000 shares in Company A for $2 per share ($2,000 total). I then negotiate a futures contract where I buy more shares in Company A, but for $50 a share. Because the stock price is dependent on what it was last traded at, if I got a shit load of futures, where I buy stock at massively inflated prices, would this then cause the stock price to rise significantly? Then I can sell my original investment for $50,000. 

Or am I just being retarded?
-----------
BUY $JNUG IT WILL SKYROCKET SHORT SQUEEZE
[removed]
-----------
I feel like I am cheating on my computer by investing in AMD. Intel gang.
[deleted]
-----------
PSA: The funding is NOT secured.
[deleted]
-----------


### Expand Tickers Column

Expand the tickers column and keep the text_hash column to relate back to the compressed comment dataframe.

In [3]:
# Expand combined_df so that each row has only one ticker

exp_file = '../../data/tabular/wsb_posts/expanded cleaned posts.csv'

if os.path.isfile(exp_file):
    print('File found, loading...')
    expanded_df = pd.read_csv(exp_file)
    

else:
    print('File not found, expanding dataframe...')
    expanded_df = pd.DataFrame(columns = ['ticker', 'text_hash'])

    for i, row in combined_df.iterrows():

        row_tickers = row.tickers.split(',')
        new_rows = pd.DataFrame({'ticker': row_tickers,
                                 'text_hash': row.text_hash})
        expanded_df = pd.concat((expanded_df, new_rows))

    expanded_df.index = np.arange(expanded_df.shape[0])
    expanded_df.to_csv(exp_file)
    
expanded_df.head()

File found, loading...


,Unnamed: 0,ticker,text_hash
0,0,QQQ,150508729389721599
1,1,AAPL,10678551544699203972
2,2,I,10730372757830112337
3,3,FB,10730372757830112337
4,4,DD,10730372757830112337


### Train/Test Split

Perform a train/test split on the text data using the combined_df table of wsb comments. We're interested in data drift over time so we'll reserve the test data to the last 5% of comments.

In [4]:
split_date = pd.to_datetime('2019-10-01')

train_df  = combined_df[pd.to_datetime(combined_df['created_utc']) < split_date]
test_df   = combined_df[pd.to_datetime(combined_df['created_utc']) >= split_date]

In [5]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import *

def process_text(string):
    # Approach is drawn from sentiment analysis project, skips
    # the HTML processing as it's not needed here.
    
    stemmer = PorterStemmer()
    words   = [w.strip() for w in string.lower().split() if w not in stopwords.words("english")]
    words   = [w for w in words if w.isalpha()]
    words   = [stemmer.stem(w) for w in words]
    
    return words

process_text(train_df.iloc[0].text)

['kind', 'option', 'play', 'would', 'use', 'profit', 'go', 'past']

In [15]:
# Apply text processing to each row of the train and test sets and
# store as dicts

train_X_file = '../../data/tabular/processed data/compressed_train_X_df.csv'
test_X_file = '../../data/tabular/processed data/compressed_test_X_df.csv'

if os.path.isfile(train_X_file) and os.path.isfile(test_X_file):
    train_X_df = pd.read_csv(train_X_file, usecols=['text_hash', 'text'])
    test_X_df = pd.read_csv(test_X_file, usecols=['text_hash', 'text'])
else:
    train_X = {row.text_hash: " ".join(process_text(row.text)) for idx, row in train_df.iterrows()}
    test_X = {row.text_hash: " ".join(process_text(row.text)) for idx, row in test_df.iterrows()}

    train_X_df = pd.DataFrame([[key, item] for key, item in train_X.items()],
                              columns = ['text_hash', 'text'])
    test_X_df = pd.DataFrame([[key, item] for key, item in test_X.items()],
                             columns = ['text_hash', 'text'])
    train_X_df.to_csv("../../data/tabular/processed data/compressed_train_X_df.csv")
    test_X_df.to_csv("../../data/tabular/processed data/compressed_test_X_df.csv")

In [44]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vectorizers = {
    'count_word': CountVectorizer(max_features=1000),
    'count_bigram': CountVectorizer(analyzer='word', ngram_range=(2, 2), max_features=1000),
    'tfidf_word': TfidfVectorizer(max_features=1000),
    'tfidf_bigram': TfidfVectorizer(analyzer='word', ngram_range=(2, 2), max_features=1000)
}

vectorizers = {key: vec.fit(train_X_df['text'].fillna('')) for key, vec in vectorizers.items()}


X_train_vec, X_test_vec = (
    {
        key: pd.concat((
            pd.DataFrame(vect.transform(x_df['text'].fillna('')).toarray(),
                         columns = vect.get_feature_names()),
            x_df['text_hash']
        ), axis=1)
        for key, vect in vectorizers.items()
    }
    for x_df in [train_X_df, test_X_df]
)

In [ ]:
vectorizers.keys()

In [ ]:
# save vectorized data
for key in vectorizers.keys():
    X_train_vec[key].to_csv('../../data/tabular/processed data/{}_train_X.csv'.format(key))
    X_test_vec[key].to_csv('../../data/tabular/processed data/{}_test_X.csv'.format(key))

In [80]:

transition_df = pd.read_csv('../../data/tabular/90th percentile transitions.csv',
                            usecols=['Date', 'Ticker', 'Transition'])
full_df = combined_df.copy()
full_df['created_utc'] = pd.to_datetime(full_df['created_utc'])
full_df['Date'] = pd.to_datetime(full_df['created_utc'].dt.date)
full_df = pd.merge(full_df, expanded_df, how='right', on='text_hash')

transition_df['Date'] = pd.to_datetime(transition_df['Date'])

full_df = (
    pd.merge(full_df, 
             transition_df, 
             left_on = ['Date', 'ticker'], 
             right_on = ['Date', 'Ticker'],
             how = 'left')
    .drop(columns = [col for col in full_df.columns if 'Unnamed' in col])
)


def apply_lag(df, lag):

    new_col = 'TransitionAfterLag'
    
    # Subtract a day from Date column to avoid information leaks
    df['Date'] = df['Date'] - pd.to_timedelta(1, unit='d')
    
    return (
        df
        .pivot(index='Date', columns=['Ticker'])
        .sort_index(ascending=False)
        .rolling(window=5)
        .max()
        .sort_index(ascending=True)
        .dropna()
        .reset_index('Date')
        .melt(id_vars='Date', value_name=new_col)
        [['Date', 'Ticker', new_col]]
    )
    
lag_transition_dfs = {lag: apply_lag(transition_df, lag) for lag in [3, 5, 10]}

lag_full_dfs = {
    lag: (
        pd.merge(full_df,
                 df,
                 left_on = ['Date', 'ticker'], 
                 right_on = ['Date', 'Ticker'],
                 how = 'left')
        .drop(columns = [col for col in full_df.columns if 'Unnmeed' in col])
    )
    for lag, df in lag_transition_dfs.items()
}



In [130]:
# save lag dfs
for lag in [3, 5, 10]:
    filename = '../../data/tabular/processed data/transition_map_lag_{}.csv'.format(lag)
    lag_full_dfs[lag][['text_hash', 'TransitionAfterLag']].to_csv(filename)

In [369]:
transitions = (
        full_df
        [['Ticker', 'Date', 'Transition']]
        [full_df['Transition'] != 0]
        .groupby(['Date', 'Ticker'])
        .count()
    )
non_transitions = (
    full_df
    [['Ticker', 'Date']]
    [full_df['Transition'] == 0]
    .groupby(['Date', 'Ticker'])
    .count()
)
pct_trans = transitions.shape[0]/non_transitions.shape[0]
print('Percent of ticker-date combinations corresponding to transitions: {:.2%}'.format(pct_trans))

Percent of ticker-date combinations corresponding to transitions: 0.37%


In [509]:
def create_examples(sample_period, 
                    n_target, 
                    n_random, 
                    text_df,
                    n_examples,
                    trans_freq=None,
                    verbose=False):
    """
    Create examples for neural net or bag-of-words models by sampling comments mentioning the
    target stock and comments mentioning other random stocks in the days leading up to a transition of
    the target into the top 10% of accounts holding. The idea is to allow the model to compare attitudes
    toward the target name with the broader market. Generates the population statistically.
    
    TODO: Test examples mode samples all transitions from the period.
    TODO: An earlier join causes comments to only be sampled from trading days. Modify such that 
          comments on weekends and holidays can also be used.
    
    @param sample_period number of days leading up to the transition date from which to
    sample comments. Does not include the date of transition. 
    @param n_target the number of comments pertaining to the target that will be included in the nn
    input
    @param n_random the number of comments to random stocks (no including the target) that will
    be included in the input
    @param text_df DataFrame containing comment text, associated tickers, and indicator of transition day
    @param verbose Boolean Indicates if the procedure will print progress and warnings to the command line.
    """
    trans_ct = (
        text_df
        [['Ticker', 'Date', 'Transition']]
        [text_df['Transition'] != 0]
        .groupby(['Date', 'Ticker'])
        .count()
    )
    non_trans_ct = (
        text_df
        [['Ticker', 'Date', 'Transition']]
        [text_df['Transition'] == 0]
        .groupby(['Date', 'Ticker'])
        .count()
    )

    examples = []
    
    if trans_freq is None:
        trans_freq = trans_ct.shape/non_trans_ct.shape
        
    for i in range(n_examples):
        select_trans_ct = np.random.rand() <= trans_freq
        
        target_pair = np.random.choice(trans_ct.index) if select_trans_ct else np.random.choice(non_trans_ct.index)
        # sample n_target from preceding sample period
        preceding_target = text_df[((text_df['created_utc'] < target_pair[0]) &
                                   (text_df['created_utc'] > target_pair[0] - pd.to_timedelta(sample_period, unit='d')) &
                                   (text_df['Ticker'] == target_pair[1]))]
        
        if preceding_target.shape[0] < n_target:
            if verbose:
                print('WARNING: Not enough target samples available for transition target, proceeding to next sample.')
            continue
            
        target_samples = np.random.choice(preceding_target['text_hash'])
        # sample n_random from preceding sample period
        
        preceding_random = text_df[(text_df['created_utc'] < target_pair[0]) &
                                   (text_df['created_utc'] >= target_pair[0] - pd.to_timedelta(sample_period, unit='d')) &
                                   ~(text_df['Ticker'] == target_pair[1])]
        if preceding_target.shape[0] <= n_random:
            if verbose:
                print('WARNING: Not enough random samples available for transition target, proceeding to next sample.')
            continue
        
        random_samples = np.random.choice(preceding_random['text_hash'], n_random)
        # append data
        example = {
            'transition': select_trans_ct,
            'date': str(target_pair[0]),
            'target': target_pair[1],
            'target_samples': target_samples,
            'random_samples': list(random_samples)
        }
        examples.append(example)
    return examples
        


sample_test_df = full_df[full_df['created_utc'] <= pd.to_datetime('2018-05-18')].copy()
test_inputs = create_examples(sample_period=5, 
                              n_target=1,
                              n_random=1,
                              text_df=sample_test_df, 
                              n_examples=5,
                              trans_freq=1.)

assert test_inputs[0]['target'] in sample_test_df.loc[sample_test_df['text_hash'] == test_inputs[0]['target_samples'], 'Ticker'].values
assert test_inputs[0]['target'] not in sample_test_df.loc[sample_test_df['text_hash'] == test_inputs[0]['random_samples'][0], 'Ticker']

In [524]:
# separate train and test temporally to avoid information leaks
split_date = pd.to_datetime('2019-10-01')

train_period_df = full_df[full_df['created_utc'] < split_date]
test_period_df = full_df[full_df['created_utc'] >= split_date]

# examine sample periods of 3, 5, and 10 days
train_examples = {
    sample_period: create_examples(sample_period=sample_period,
                                   n_target=1,
                                   n_random=3,
                                   text_df=train_period_df,
                                   n_examples=5000,
                                   trans_freq=0.5) for sample_period in [3, 5, 10]
}

test_examples = {
    sample_period: create_examples(sample_period=sample_period,
                                   n_target=1,
                                   n_random=3,
                                   text_df=test_period_df,
                                   n_examples=1000,
                                   trans_freq=0.1) for sample_period in [3, 5, 10]
}


print('Number of train/test examples for 3 trailing days: {}/{}'.format(len(train_examples[3]), len(test_examples[3])))
print('Number of train/test examples for 5 trailing days: {}/{}'.format(len(train_examples[5]), len(test_examples[5])))
print('Number of train/test examples for 10 trailing days: {}/{}'.format(len(train_examples[10]), len(test_examples[10])))

Number of train/test examples for 3 trailing days: 785/153
Number of train/test examples for 5 trailing days: 1046/195
Number of train/test examples for 10 trailing days: 1404/291


In [525]:
def get_true_frac(example_list):
    n_example = len(example_list)
    n_trans = sum([example['transition'] for example in example_list])
    n_nontrans = n_example - n_trans
    return float(n_trans)/float(n_example)
    
for i in [3, 5, 10]:
    print('Percet Transitions {} Trailing Days Train: {:.2%}'.format(i, get_true_frac(train_examples[i])))
    print('Percet Transitions {} Trailing Days Test: {:.2%}'.format(i, get_true_frac(test_examples[i])))


Percet Transitions 3 Trailing Days Train: 56.43%
Percet Transitions 3 Trailing Days Test: 5.88%
Percet Transitions 5 Trailing Days Train: 47.42%
Percet Transitions 5 Trailing Days Test: 6.15%
Percet Transitions 10 Trailing Days Train: 44.44%
Percet Transitions 10 Trailing Days Test: 7.22%


In [526]:
for i in [3, 5, 10]:
    with open("../../data/heirarchical/processed data/{}-day-example-map-train.json".format(i), "w") as json_file:
        json.dump(train_examples[i], json_file)
    with open("../../data/heirarchical/processed data/{}-day-example-map-test.json".format(i), "w") as json_file:
        json.dump(test_examples[i], json_file)

In [556]:
train_examples[3][0]['target_samples'] in train_X_df['text_hash'].values

True

In [552]:
train_examples[3][0]['target_samples']

14394462476021379538